# What Makes A Best Original Screenplay?

    By Allison Ragan and Allison Shafer
    American University
    STAT-696 Applied Natural Language Processing

# Analysis

# Table of Contents

    I. Package Imports
    II. NLP Transformations
        i. CountVectorized
        ii. TFIDF
    III. Sentiment Analysis
        i. Sentiment Analysis -- Dialogue
        ii. Sentiment Analysis -- Full Script
    IV. Topic Modeling
        i. Topic Modeling -- Dialogue
        ii. Topic Modeling -- Full Script
    V. K-Means Clustering
        i. K-Means Clustering -- Dialogue
        ii. K-Means Clustering -- Full Script
    VI. Results
    VII. Deprecated Code

## Package Imports

In [84]:
import pandas as pd 
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF 
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.decomposition import NMF as nmf
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import wordcloud # may need to install
from plotnine import * # may need to install; ggplot included
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from sklearn.manifold import TSNE

from afinn import Afinn


In [11]:
# Read in .csv to dataframe

movies_df = pd.read_csv('./movies_df.csv')
movies_df

,Unnamed: 0,Title,Script,year,orig_screen_win,best_pic_win,reviewer,cleaned_dialogue,cleaned_script,script_length,dialogue_length,Genres,GenresBroad,Rating,Critic_Score,Aud_Score,RunTime
0,0,First_Reformed,Scripts.com\n\nFirst Reformed\n\nBy Paul Schra...,2018,0,NaN,Ragan,keep journal word program digital file longhan...,bird calling distance toller keep journal word...,4362,4310,Drama,Drama,R,93,68,108
1,1,GreenBook,GREEN BOOK \n\nWritten by \n\nNick Vallelonga ...,2018,1,Win,Ragan,thank much welcome jules podell copacabana bob...,fade title card new york city 1962 ext copacab...,12731,7640,Drama,Drama,PG-13,78,91,130
2,2,Hell_or_HighWater,Scripts.com\n\nHell or High Water\n\nBy Taylor...,2016,0,Nom,Ragan,quiet open door open door devil please stand t...,quiet open door open door devil please stand t...,3627,3627,Drama,Mystery Suspense,R,97,88,102
3,3,LadyBird,\n \n\n \n \n \n \n \n \n \n \n \n \n \n \n \...,2017,0,Nom,Ragan,making bed looking back loan scholarship world...,anybody talk california hedonism ha never spe...,10058,6237,Comedy-Drama,Comedy-Drama,R,99,79,93
4,4,LaLaLand,LA LA LAND\n\nby\n\nDamien Chazelle\n\n A sun-...,2016,0,Nom,Ragan,think going worst working like gon na talk tal...,sunblasted sky hear radio one piece music ano...,9817,6853,Comedy-Drama,Comedy-Drama,PG-13,91,81,128
5,5,MarriageStory,MARRIAGE STORY\n\nWritten and Directed by Noah...,2019,0,Nom,Ragan,make people feel comfortable even embarrassing...,charlie love nicole int theater manhattan day ...,14570,10621,Drama,Drama,R,95,85,136
6,6,Roma,IN ENGLISH\n\n ROMA\n\nWritten and Directed by...,2018,0,Nom,Ragan,said bed look mommy grade watermelon cleo coul...,int patio tepeji 21 day yellow triangle inside...,41539,31266,Drama,Drama,R,95,72,135
7,7,ShapeofWater,F O R Y O U R C O N S I D E R A T I O N\n\...,2017,0,Win,Ragan,spoke it did would tell wonder would tell time...,int elisa apartment river footage night float ...,24104,17253,Drama,Drama,R,92,72,119
8,8,TheBigSick,THE BIG SICK\n\nby\nEmily V. Gordon & Kumail N...,2017,0,NaN,Ragan,bob dalaban true cokehead gon na say anything ...,black hear audience laughter andy dodd give ma...,10618,7508,Comedy-Romance,Comedy,R,98,88,119
9,9,TheFavourite,F O R Y O U R C O N S I D E R A T I O N\n\...,2018,0,Nom,Ragan,mud stink lady marlborough yes like lisp love ...,ext countryside carriage cross landscape int c...,11112,6967,Drama,Drama,R,93,69,119


## Data Visualizations

## NLP transformations

### CV

In [20]:
# instantiate

# all scripts and visuals 
bow_all = CountVectorizer(ngram_range=(1,2), max_df = .85, min_df = .2)
# fit + transform training data
bow_all_fit = bow_all.fit_transform(movies_df['cleaned_script'])

# for dialogue only
bow_dialogue = CountVectorizer(ngram_range=(1,2), max_df = .85, min_df = .2)
# fit + transform training data
bow_dialogue_fit = bow_dialogue.fit_transform(movies_df['cleaned_dialogue'])

### TFIDF

In [28]:
# instantiate
tfidf_all = TfidfVectorizer(ngram_range = (1,2), max_df = .85, min_df = .2, stop_words = 'english')

# fit + transform training data - all scripts and visuals

tfidf_all_fit = tfidf_all.fit(movies_df['cleaned_script'])
tfidf_vocab_all = tfidf_all_fit.vocabulary_
tfidf_vocab_all = {k: v for k, v in sorted(tfidf_vocab_all.items(), key=lambda item: item[1], reverse=True)}
print(tfidf_vocab_all)

{'zone': 3841, 'zero': 3840, 'yup': 3839, 'youve': 3838, 'youre right': 3837, 'youre going': 3836, 'youre': 3835, 'younger': 3834, 'young woman': 3833, 'young man': 3832, 'youll': 3831, 'youd': 3830, 'york city': 3829, 'york': 3828, 'yesterday': 3827, 'yes yes': 3826, 'yes wa': 3825, 'yes think': 3824, 'yes thank': 3823, 'yes sir': 3822, 'yes mr': 3821, 'yes know': 3820, 'yes int': 3819, 'yes course': 3818, 'yep': 3817, 'yellow': 3816, 'yelling': 3815, 'yell': 3814, 'yearold': 3813, 'year wa': 3812, 'year old': 3811, 'year later': 3810, 'year ago': 3809, 'yeah yeah': 3808, 'yeah wa': 3807, 'yeah thats': 3806, 'yeah sure': 3805, 'yeah sorry': 3804, 'yeah right': 3803, 'yeah okay': 3802, 'yeah mean': 3801, 'yeah like': 3800, 'yeah know': 3799, 'yeah int': 3798, 'yeah im': 3797, 'yeah guess': 3796, 'yeah got': 3795, 'yeah good': 3794, 'yay': 3793, 'yawn': 3792, 'yard away': 3791, 'yard': 3790, 'yank': 3789, 'ya': 3788, 'wrote': 3787, 'written': 3786, 'writing': 3785, 'writes': 3784, 'writ

In [27]:
# instantiate
tfidf_dialogue = TfidfVectorizer(ngram_range = (1,2), max_df = .85, min_df = .2, stop_words = 'english')

# fit + transform training data - dialogue only 
tfidf_dialogue_fit = tfidf_dialogue.fit_transform(movies_df['cleaned_dialogue'])
tfidf_vocab_dialogue = tfidf_dialogue.vocabulary_
tfidf_vocab_dialogue = {k: v for k, v in sorted(tfidf_vocab_dialogue.items(), key=lambda item: item[1], reverse=True)}
print(tfidf_vocab_dialogue)

{'zero': 2919, 'yup': 2918, 'youve': 2917, 'youre going': 2916, 'youre': 2915, 'younger': 2914, 'young woman': 2913, 'young man': 2912, 'youll': 2911, 'youd': 2910, 'york city': 2909, 'york': 2908, 'yesterday': 2907, 'yes yes': 2906, 'yes wa': 2905, 'yes think': 2904, 'yes thank': 2903, 'yes sir': 2902, 'yes mr': 2901, 'yes know': 2900, 'yes good': 2899, 'yes course': 2898, 'yep': 2897, 'yellow': 2896, 'yelling': 2895, 'yell': 2894, 'yearold': 2893, 'year old': 2892, 'year ago': 2891, 'yeah yeah': 2890, 'yeah wa': 2889, 'yeah sorry': 2888, 'yeah right': 2887, 'yeah oh': 2886, 'yeah need': 2885, 'yeah mean': 2884, 'yeah like': 2883, 'yeah know': 2882, 'yeah guess': 2881, 'yeah got': 2880, 'yeah good': 2879, 'yeah fuck': 2878, 'yard': 2877, 'yank': 2876, 'ya': 2875, 'wrote': 2874, 'wrong': 2873, 'written': 2872, 'writing': 2871, 'writes': 2870, 'writer': 2869, 'write': 2868, 'wrist': 2867, 'wrestling': 2866, 'wrapping': 2865, 'wrapped': 2864, 'wrap': 2863, 'wow': 2862, 'wounded': 2861, '

# Sentiment Analysis

Using AFINN lexicon, aka the "new ANEW" which calculates a polarity score associated with each word.


### Sentiment Analysis -- Dialogue

In [82]:
# initialize afinn sentiment analyzer
af = Afinn()

# compute sentiment scores (polarity) and labels
sentiment_scores = [af.score(article) for article in list(movies_df['cleaned_dialogue'])]
sentiment_category = ['positive' if score > 0 
                          else 'negative' if score < 0 
                              else 'neutral' 
                                  for score in sentiment_scores]

afinn_dialogue = pd.DataFrame([list(movies_df['Title']), list(movies_df['Genres']), list(movies_df['GenresBroad']), list(movies_df['orig_screen_win']), sentiment_scores, sentiment_category]).T
afinn_dialogue.columns = ['Titles', 'Genres', 'GenresBroad', 'Winner', 'SentimentScore', 'SentimentCategory']
afinn_dialogue['SentimentScore'] = afinn_dialogue['SentimentScore'].astype('float')

NameError: name 'Afinn' is not defined

In [ ]:
plt.figure(figsize=(14,4))
sns.countplot(x='SentimentCategory', data=afinn_dialogue, palette="hls").set_title("Sentiment Score Counts")
plt.show()

#### Winners vs Losers

In [ ]:
# sentiment statistics for winners/losers
afinn_dialogue.groupby(by=['Winner']).describe()

In [ ]:
# boxplot
fig_dims = (20, 8)
fig, ax = plt.subplots(figsize=fig_dims)
sns.boxplot(x='Winner', y="SentimentScore", 
                 hue='Winner', data=afinn_dialogue, ax=ax,
                 palette="hls").set_title('Boxplot of Sentiment Score by Result')

In [ ]:
# barplot
fig_dims = (20, 8)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x = "Winner", y = "SentimentScore", ax=ax, data=afinn_dialogue, 
            palette="hls", ci=None).set_title('Bar Plot of Sentiment Score by Result')

In [ ]:
# countplot
fig_dims = (20, 8)
fig, ax = plt.subplots(figsize=fig_dims)
sns.countplot(y="Winner", hue="SentimentCategory", data=afinn_dialogue, 
              ax=ax, palette="hls").set_title('Count Plot of Sentiment Score by Result')

### Sentiment Analysis -- Full Script

In [ ]:
# compute sentiment scores (polarity) and labels
sentiment_scores = [af.score(article) for article in list(movies_df['cleaned_script'])]
sentiment_category = ['positive' if score > 0 
                          else 'negative' if score < 0 
                              else 'neutral' 
                                  for score in sentiment_scores]

afinn_all = pd.DataFrame([list(movies_df['Title']), list(movies_df['Genres']), list(movies_df['GenresBroad']), list(movies_df['orig_screen_win']), sentiment_scores, sentiment_category]).T
afinn_all.columns = ['Titles', 'Genres', 'GenresBroad', 'Winner', 'SentimentScore', 'SentimentCategory']
afinn_all['SentimentScore'] = afinn_all['SentimentScore'].astype('float')

## Topic Modeling -- AS

### TFIDF/ NMF

### Topic Modeling -- Dialogue

In [69]:
# instantiate
tfidf_dialogue = TfidfVectorizer(ngram_range = (1,2), max_df = .85, min_df = .2, stop_words = 'english')

# fit + transform training data - dialogue only 
tfidf_dialogue_fit = tfidf_dialogue.fit_transform(movies_df['cleaned_dialogue'])
tfidf_vocab_dialogue = tfidf_dialogue.vocabulary_
# tfidf_vocab_dialogue = {k: v for k, v in sorted(tfidf_vocab_dialogue.items(), key=lambda item: item[1], reverse=True)}
# print(tfidf_vocab_dialogue)

tf_diag_feat = tfidf_dialogue.get_feature_names()
len(tf_diag_feat)

{'zero': 2919, 'yup': 2918, 'youve': 2917, 'youre going': 2916, 'youre': 2915, 'younger': 2914, 'young woman': 2913, 'young man': 2912, 'youll': 2911, 'youd': 2910, 'york city': 2909, 'york': 2908, 'yesterday': 2907, 'yes yes': 2906, 'yes wa': 2905, 'yes think': 2904, 'yes thank': 2903, 'yes sir': 2902, 'yes mr': 2901, 'yes know': 2900, 'yes good': 2899, 'yes course': 2898, 'yep': 2897, 'yellow': 2896, 'yelling': 2895, 'yell': 2894, 'yearold': 2893, 'year old': 2892, 'year ago': 2891, 'yeah yeah': 2890, 'yeah wa': 2889, 'yeah sorry': 2888, 'yeah right': 2887, 'yeah oh': 2886, 'yeah need': 2885, 'yeah mean': 2884, 'yeah like': 2883, 'yeah know': 2882, 'yeah guess': 2881, 'yeah got': 2880, 'yeah good': 2879, 'yeah fuck': 2878, 'yard': 2877, 'yank': 2876, 'ya': 2875, 'wrote': 2874, 'wrong': 2873, 'written': 2872, 'writing': 2871, 'writes': 2870, 'writer': 2869, 'write': 2868, 'wrist': 2867, 'wrestling': 2866, 'wrapping': 2865, 'wrapped': 2864, 'wrap': 2863, 'wow': 2862, 'wounded': 2861, '

2920

In [81]:
n = nmf(n_components = 10).fit(tfidf_dialogue_fit) # set up nmf model for 10 topics - one for each movie?

# function to marry up word value to feature names and output words per topic
def print_top_words(model, feature_names, n_top_words): 
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic %d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

print_top_words(n, tf_diag_feat, 20) # 20 words per topic



n = nmf(n_components = 24).fit(tfidf_dialogue_fit) # set up nmf model for 24 topics - one for each movie?

# function to marry up word value to feature names and output words per topic
def print_top_words(model, feature_names, n_top_words): 
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic %d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()

print_top_words(n, tf_diag_feat, 20)

Topic 0: beat sir nod pie egg sign blood grab german corridor record men alright suddenly push glass god club wall camera
Topic 1: dont im youre thats david shes didnt whats doesnt dont know ill william im sorry theyre ok youll id wont ive im going
Topic 2: na gon gon na fucking uh bank beth fuck ca wan wan na god um pause ya cube em cop bitch ok
Topic 3: la charlie los al sofa new york york ok theater ya le mom patio ms hesitates lawyer movie envelope divorce costume
Topic 4: lady bird mom mr dad suddenly nod shall horse college rabbit dress happy war mail hurt peace fall father cunt
Topic 5: dick power president mary agent war american state mr beat judge secretary government soldier office united plane men bush legal
Topic 6: rose rod picture brooklyn driver dont lawn dining im richard living room begin jim chair doesnt thats american detective missing fall
Topic 7: lip dr christmas piano record glance na gon gon na george charlie hell stage letter chief mr eat road stone money
Topi

### Topic Modeling -- Full Script

In [76]:
# instantiate
tfidf_all = TfidfVectorizer(ngram_range = (1,2), max_df = .8, min_df = .25, stop_words = 'english')

# fit + transform training data - all scripts and visuals
tfidf_all_fit = tfidf_all.fit_transform(movies_df['cleaned_script'])
#tfidf_vocab_all = tfidf_all_fit.vocabulary_
#tfidf_vocab_all = {k: v for k, v in sorted(tfidf_vocab_all.items(), key=lambda item: item[1], reverse=True)}
#print(tfidf_vocab_all)


tf_all_feat = tfidf_all.get_feature_names()
len(tf_all_feat)

2915

In [78]:
n = nmf(n_components = 10).fit(tfidf_all_fit) # set up nmf model for 10 topics

# function to marry up word value to feature names and output words per topic
def print_top_words(model, feature_names, n_top_words): 
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic %d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
    
print_top_words(n, tf_all_feat, 10)


n = nmf(n_components = 24).fit(tfidf_all_fit) # set up nmf model for 24 topics

# function to marry up word value to feature names and output words per topic
def print_top_words(model, feature_names, n_top_words): 
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic %d: " % topic_idx
        message += " ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)
    print()
    
print_top_words(n, tf_all_feat, 10)

Topic 0: cont apartment richard pie party os mr shooting sir corridor
Topic 1: contd dont im youre thats william power doesnt shes rose
Topic 2: na gon gon na mary bank cube um ca leaning fucking
Topic 3: charlie james os police station continuous police ya pause fucking road
Topic 4: lip dr cont tony george continuous joe ok stage apartment
Topic 5: lady bird cont mom mr shooting 14 os sister college
Topic 6: dick david bush president hotel waiter george mary manager agent
Topic 7: la sofa los cont patio cube da le ya record
Topic 8: joy sadness memory fear anger mom dad island contd san
Topic 9: mike im jim joe church dont thats law contd cube

Topic 0: cont pie corridor sir egg apartment general contd van cart
Topic 1: contd im dont youre corridor monitor study power screen mountain
Topic 2: na bank gon gon na branch brother maam drawer son bitch fucking
Topic 3: charlie os ok la new york york theater frank mom female
Topic 4: lip dr tony cont george continuous stage os christmas ro

## K Means Clustering  -- AR

Either on CV and TFIDF, or just on NMF

### K Means Clustering -- Dialogue

#### CV

#### TFIDF

### K Means Clustering -- Full Script

#### CV

#### TFIDF

In [ ]:
# from homework 6

# def eval_k(kmeans, centroids, terms):
#     cluster_terms = []
#     for i in range(kmeans.n_clusters):
#         termss = []
#         for ind in centroids[i, :10]:
#             termss.append(terms[ind])
#         cluster_terms.append(termss)
#     for i in range(kmeans.n_clusters):
#         print(f'\tCluster {i+1}: {" ".join(cluster_terms[i])}')

# # K-Means with a) Count Vectorized
# k_cv = KMeans(n_clusters = 20, random_state = 0).fit(cv_tf)
# k_cv_centroids = k_cv.cluster_centers_.argsort()[:, ::-1]
# k_cv_terms = cv_feat_names
# k_cv_labels = k_cv.labels_

# print("Count Vectorized:")
# eval_k(k_cv, k_cv_centroids, k_cv_terms)

# # K-Means with b) TFIDF
# k_tfidf = KMeans(n_clusters = 20, random_state = 0).fit(tfidf)
# k_tfidf_centroids = k_tfidf.cluster_centers_.argsort()[:, ::-1]
# k_tfidf_terms = cv_feat_names
# k_tfidf_labels = k_tfidf.labels_

# print("TFIDF:")
# eval_k(k_tfidf, k_tfidf_centroids, k_tfidf_terms)

# # K-Means with c) NMF -- (1,1), (1,2), (2,3) based on 4
# k_nmf = KMeans(n_clusters = 20, random_state = 0).fit(tfidf_onetwo_matrix)
# k_nmf_centroids = k_nmf.cluster_centers_.argsort()[:, ::-1]
# k_nmf_terms = tfidf_onetwo_feat_names
# k_nmf_labels = k_nmf.labels_

# print("NMF:")
# eval_k(k_nmf, k_nmf_centroids, k_nmf_terms)